[![](https://bytebucket.org/davis68/resources/raw/f7c98d2b95e961fae257707e22a58fa1a2c36bec/logos/baseline_cse_wdmk.png?token=be4cc41d4b2afe594f5b1570a3c5aad96a65f0d6)](http://cse.illinois.edu/)

# C & Fortran Interfacing with Python

## Contents
- [Python as Duct Tape](#motivation)
- [Fortran](#fortran)
    - [A Brief Introduction](#intro-f)
    - [F2Py](#f2py)
- [C](#c)
    - [A Brief Introduction](#intro-c)
    - [Using C with Python](#using-c)
    - [Weave](#weave)
    - [Cython](#cython)
- [References](#refs)
- [Credits](#credits)

![](http://fc03.deviantart.net/fs70/f/2012/316/e/1/duct_tape_force_quote_by_seekerarmada-d5ktpyq.jpg)

<a id='motivation'></a>
## Python as Duct Tape

Python is considered to behave as an excellent adhesive between different programming languages.  Most of what we have seen along those lines thus far involves invoking shell programs indirectly, perhaps via IPython's `!` notation or by using `sys` or `os.popen`.  However, many performance-critical modules in Python, such as NumPy, have substantial components written in C or other high-performance languages.  We'll examine several ways to interface or incorporate other languages—particularly C and Fortran—into your Python program in this lesson.

In [ ]:
from __future__ import division

import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
import sys
print(sys.prefix)

!echo $PYTHONPATH

**Motivation**

As is probably now clear to you, Python is a flexible and easy-to-use language. The programmer time required to program a new model or calculation is typically quite short.

However, this reduction in human time, comes at the expense of increasing computation time, especially when we are talking about calculations which involve iterating over large arrays. Interpreted languages (MATLAB, Python, Perl, Ruby, etc.) are thus categorically slower than low-level compiled languages (C, C++, Fortran, etc.).  On the other hand, writing lengthy programs in C and FORTRAN is a cumbersome task. This tradeoff is one motivation to code the expensive chunks in C or FORTRAN and glue the bigger picture with Python.

![](http://www.behnel.de/cython200910/SimplicityVsSpeed1.png)

In real terms, the range in relative performance on a set of standard benchmarks can be quantified thus (where 1.0 is the performance of C):

[![](https://bytebucket.org/davis68/python/raw/310d4816ada46e9efd3593d45e6df8b46d2b66fc/lessons/img/benchmarks.png?token=43e6fd87f223127266c46c3a48325350224a73a0)](http://julialang.org/benchmarks/)

Python doesn't comport itself poorly among these peers, but it's still not in the same performance league as the heavy hitters.

Another major reason you may need to use Fortran or C with Python is access to legacy code:  code that is already performing to specifications that would be too expensive, difficult, or loss-incurring to port to Python.

---
<a id='fortran'></a>
## Fortran

<a id='intro-f'></a>
### A Brief Introduction to Fortran

[Fortran](https://en.wikipedia.org/wiki/Fortran) (**For**mula **Tran**slation) was introduced in 1954 for numeric and scientific computing.  Fortran is compiled to machine code for the purpose of speed, and it serves as the benchmark for high-performance computing even today.  Despite ongoing criticisms, it has managed to survive and adapt for nearly sixty years.  Fortran still exists and dominates HPC due to both its computational performance and *software inertia*—the enormously large body of legacy code written in Fortran.

Let's look at an example FORTRAN function to get a better understanding of how it works!

_Note:  This lesson's aim is not to verse you well in Fortran, so we'll neglect nitty-gritty details.  We will introduce just enough to address common needs and demonstrates interfacing with Python._

In [ ]:
%%file circle.f
PROGRAM CIRCLE
REAL R, AREA
! This program reads a real number r and prints
! the area of a circle with radius r.
WRITE (*,*) 'Enter radius, r = '
READ (*,*) R
AREA = 3.14159*R*R
WRITE (*,*) 'Area, A = ', AREA
STOP
END

The resulting compilation and execution:

    $ gfortran -ffree-form -o circle circle.f
    $ ./circle
     Enter radius, r = 
    1
     Area, A =    3.14159012 

Anyway, that's a little ugly, isn't it?  Originally, all Fortran programs had to be written in upper-case letters.  Most people now write lower-case.  You are free to mix letter case, but note that Fortran is not case-sensitive:  `X` and `x` are the same variable.

If you wish to use $\pi$ as a constant, you can do it using the `parameter` statement:

In [ ]:
%%file circle.f
program circle
real r, area, pi
parameter (pi = 3.14159)
! This program reads a real number r and prints
! the area of a circle with radius r.
write (*,*) 'Enter radius, r = '
read (*,*) R
area = 3.14159*r*r
write (*,*) 'Area, A = ', area
stop
end

#### Data Types

Every variable _should_ be defined in a declaration.  This establishes the type of the variable. The most common declarations are:

      integer              list of variables
      real                 list of variables
      double precision     list of variables
      complex              list of variables
      logical              list of variables
      character            list of variables

Implicit typing can be turned on or off using `implicit [real]` or `implicit none`, and the latter in particular is often seen in the wild.

- [FORTRAN77 Tutorial](http://web.stanford.edu/class/me200c/tutorial_77/10_arrays.html)

#### Arrays

FORTRAN uses one dimensional arrays to correspond to vectors and 2D arrays to represent matrices.

##### 1D Arrays

    real a(20)

declares an array of size 20.  By convention, Fortran arrays are indexed from 1 (MATLAB follows this convention as well).  Thus the first number in the array is denoted by a(1) and the last by a(20).

    integer i(10)
    logical aa(0:1)
    double precision x(100)

This code segment stores the 10 first square numbers in the array sq:

    integer i, sq(10)
    do 100 i = 1, 10
         sq(i) = i**2
    100 continue
  
A common bug in Fortran is that the program tries to access array elements that are out of bounds or undefined.  This is the responsibility of the programmer, and the Fortran compiler will not detect any such bugs!

_Note_:  One tricky thing about reading Fortran code is that constructs may not mean what you think.  For instance:
- `real*8 a` does not mean a vector `a` with eight components, but an eight-byte (C `double`) floating-point number `a`.
- `.le.` and other operators of this type mean less than or equal to, `<=`, etc.

##### 2D Arrays

    real A(3,5)

declares a 2D array of size 3x5.  It is common to declare arrays of a larger size than what we use because in FORTRAN we cannot dynamically change the size of an array.  In other words, arrays are *static*.

    real A(3,5)
    integer i,j
    ! We will only use the upper 3 by 3 part of this array.
    do 20 j = 1, 3
      do 10 i = 1, 3
          a(i,j) = real(i)/real(j)
      10 continue
    20 continue

Note also that, contrary to C and Python, Fortran arrays are _column-major_.  The first index refers to the _row_, as is conventional in linear algebraic notation, but loops would need to be reversed relative to C (thus loop over rows before columns in Fortran for efficiency ([src](http://jblevins.org/log/efficient-code))).  This becomes critical when passing data to and from Fortran.

---
<a id='f2py'></a>
#### F2PY

The [F2PY](http://docs.scipy.org/doc/numpy-dev/f2py/) program (now part of NumPy) offers the best of both worlds:  it wraps existing Fortran code for use in Python, including providing reasonable documentation automatically.  With F2PY, we can compile modules from Fortran code which can then be imported into Python directly.



- [F2PY References](http://www.f2py.com/home/references)

In [ ]:
%%file hellofortran.f
! File  hellofortran.f
        subroutine hellofortran (n)
        integer n
        
        do 100 i=0, n
            print *, "Fortran says hello"
100     continue
        end

In [ ]:
!f2py3.4 -c -m hellofortran hellofortran.f

Now we write a Python script which uses the module

In [ ]:
%%file hello.py
import hellofortran

hellofortran.hellofortran(5)

In [ ]:
# run the script
!python hello.py

*Vector input , scalar output *

In [ ]:
%%file dprod.f

       subroutine dprod(x, y, n)
    
       double precision x(n), y
       y = 1.0
    
       do 100 i=1, n
           y = y * x(i)
100    continue
       end


In [ ]:
!rm -f dprod.pyf
!f2p3.4y -m dprod -h dprod.pyf dprod.f

The f2py program generates a declaration file `dprod.pyf`

In [ ]:
!cat dprod.pyf

The module does not know what Fortran subroutine arguments is input and output, so we need to manually edit the module declaration files and mark output variables with intent(out) and input variable with intent(in):

In [ ]:
%%file dprod.pyf
python module dprod ! in 
    interface  ! in :dprod
        subroutine dprod(x,y,n) ! in :dprod:dprod.f
            double precision dimension(n), intent(in) :: x
            double precision, intent(out) :: y
            integer, optional,check(len(x)>=n),depend(x),intent(in) :: n=len(x)
        end subroutine dprod
    end interface 
end python module dprod

In [ ]:
!f2py3.4 -c dprod.pyf dprod.f  #Compile the code which can be included in Python

In [ ]:
import dprod #Using the module in Python

In [ ]:
help (dprod)

In [ ]:
dprod.dprod(np.arange(1,50))

In [ ]:
np.prod(np.arange(1.0,50.0))

Comparing the performance

In [ ]:
xvec = np.random.rand(1000)

In [ ]:
%timeit dprod.dprod(xvec)

In [ ]:
%timeit xvec.prod()

##### Example:  Cumulative sum of an array

The cumulative sum over data in an array is a loop-intensive algorithm and hence a good example to exploit the performance advantage in Fortran.

In [ ]:
# pure Python 
def py_dcumsum(a):
    b = np.empty_like(a)
    b[0] = a[0]
    for n in range(1,len(a)):
        b[n] = b[n-1]+a[n]
    return b

In [ ]:
%%file dcumsum.f
c File dcumsum.f
       subroutine dcumsum(a, b, n)
       double precision a(n)
       double precision b(n)
       integer n
cf2py  intent(in) :: a
cf2py  intent(out) :: b
cf2py  intent(hide) :: n

       b(1) = a(1)
       do 100 i=2, n
           b(i) = b(i-1) + a(i)
100    continue
       end

In [ ]:
!f2py3.4 -c dcumsum.f -m dcumsum

In [ ]:
import dcumsum

a = np.array([1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0])

In [ ]:
py_dcumsum(a)

In [ ]:
dcumsum.dcumsum(a)

In [ ]:
a = np.random.rand(10000)

In [ ]:
%timeit py_dcumsum(a)

In [ ]:
%timeit dcumsum.dcumsum(a)

---
<a id='c'></a>
## C

Now we move on to a more intuitive—or at least familiar—language C.  C is often called a _mid-level_ or _systems_ programming language.  This is not a reflection on its lack of programming power but more a reflection on its capability to access the system's low level functions.  It wins on the performance aspect and also on the not-as-ludicrously-obscure-as-Fortran aspect.

Despite its increased ease of use as opposed to FORTRAN, it still requires quite a bit of detail with respect to data types, functions, data structures etc.  Python still involves less human time than C does.  So, we benefit from gluing C and Python together.

---
<a id='intro-c'></a>
#### A Brief Introduction to C

##### Data Types

- `int`—integer.
- `float`—floating point value:  _i.e._, a number with a fractional part.
- `double`—a double-precision floating point value.
- `char`—a single character.
- `void`—a valueless special-purpose type.

##### Input/Output Functions

- [`printf`](http://www.cplusplus.com/reference/cstdio/printf/)—Output formatted string.
- [`scanf`](http://www.cplusplus.com/reference/cstdio/scanf/)—Input stream (until Return pressed).
- [`getchar`](http://www.cplusplus.com/reference/cstdio/getchar/)—Input single character.

##### A Simple Program in C
(This program should be run at the command line; `getchar`-based input is not currently supported in the IPython notebook interface.)

Salient features of this program include the `#include` statement, the `int` return type of the `main` function, and the use of `{` and `}` to delimit a block of code.

In [ ]:
%%file test.c
#include <stdio.h>
int main() {
    printf( "Happy Halloween! \n" );
    getchar();
    return 0;
}

In [ ]:
!gcc -o test test.c

In [ ]:
!./test

##### Arrays
    int some_array[5];
declares an array of five elements.

###### Example:  Array Summation

    int sum_values_of_array(int all_nums[]) {
        int i, sum=0;
        for(i = 0; i<5; i++)
            sum = sum + all_nums[i];
        return sum;
    }

---
<a id='ctypes'></a>
#### `ctypes`

[`ctypes`](https://docs.python.org/3/library/ctypes.html) is a Python library for calling out to C libraries.  It is not as automatic as F2PY, and we manually need to load the library and set properties such as the function argument and return types.  On the other hand, we do not need to touch the C code at all.

In [ ]:
import ctypes
ctypes.cdll.LoadLibrary("libc.so.6")
libc = ctypes.CDLL("libc.so.6")
libc.printf

In [ ]:
print(libc.time(None)) 

---

In [ ]:
%%file functions.c

#include <stdio.h>

void hello(int n);

double dprod(double *x, int n);

void dcumsum(double *a, double *b, int n);

void
hello(int n)
{
    int i;
    
    for (i = 0; i < n; i++)
    {
        printf("C says hello\n");
    }
}


double 
dprod(double *x, int n)
{
    int i;
    double y = 1.0;
    
    for (i = 0; i < n; i++)
    {
        y *= x[i];
    }

    return y;
}

void
dcumsum(double *a, double *b, int n)
{
    int i;
    
    b[0] = a[0];
    for (i = 1; i < n; i++)
    {
        b[i] = a[i] + b[i-1];
    }
}

Compiling the C code

In [ ]:
!gcc -c -Wall -O2 -Wall -ansi -pedantic -fPIC -o functions.o functions.c
!gcc -o libfunctions.so -shared functions.o

In [ ]:
!file libfunctions.so

Now we need to write wrapper functions to access the C library: To load the library we use the ctypes package, which included in the Python standard library (with extensions from numpy for passing arrays to C). Then we manually set the types of the argument and return values (no automatic code inspection here!). 

In [ ]:
%%file functions.py

import numpy
import ctypes

_libfunctions = numpy.ctypeslib.load_library('libfunctions', '.')

_libfunctions.hello.argtypes = [ctypes.c_int]
_libfunctions.hello.restype  =  ctypes.c_void_p

_libfunctions.dprod.argtypes = [numpy.ctypeslib.ndpointer(dtype=numpy.float), ctypes.c_int]
_libfunctions.dprod.restype  = ctypes.c_double

_libfunctions.dcumsum.argtypes = [numpy.ctypeslib.ndpointer(dtype=numpy.float), numpy.ctypeslib.ndpointer(dtype=numpy.float), ctypes.c_int]
_libfunctions.dcumsum.restype  = ctypes.c_void_p

def hello(n):
    return _libfunctions.hello(int(n))

def dprod(x, n=None):
    if n is None:
        n = len(x)
    x = numpy.asarray(x, dtype=numpy.float)
    return _libfunctions.dprod(x, int(n))

def dcumsum(a, n):
    a = numpy.asarray(a, dtype=numpy.float)
    b = numpy.empty(len(a), dtype=numpy.float)
    _libfunctions.dcumsum(a, b, int(n))
    return b

In [ ]:
%%file run_hello_c.py

import functions

functions.hello(3)

In [ ]:
!python run_hello_c.py

In [ ]:
import functions

In [ ]:
functions.dprod([1,2,3,4,5]) #product

In [ ]:
a = np.random.rand(100000)

In [ ]:
res_c = functions.dcumsum(a, len(a)) 

In [ ]:
res_fortran = dcumsum.dcumsum(a)

In [ ]:
res_c - res_fortran

###### Timing

In [ ]:
%timeit functions.dcumsum(a, len(a))

In [ ]:
%timeit dcumsum.dcumsum(a)

In [ ]:
%timeit a.cumsum()

---
<a id='ctypes'></a>
#### Weave

[Weave](http://docs.scipy.org/doc/scipy-0.14.0/reference/tutorial/weave.html) essentially does for C/C++ what F2PY does for Fortran:  we can use data stored in NumPy `ndarray`s with C/C++ code for mission-critical performance.  (For instance, C can unroll loops or recurse faster than Python.)

Weave is specific to Python 2<sup>[[ref](https://github.com/scipy/scipy/issues/3171)]</sup>, and thus is not always installed with SciPy.  The following code may not work on your machine since we are teaching in Python 3.

##### Example:  Laplace Equation

Consider solving the Laplace equation $\nabla^2 u = 0$ over a square grid ([source](http://technicaldiscovery.blogspot.com/2011/06/speeding-up-python-numpy-cython-and.html)).  The pure Python version follows:

In [ ]:
from numpy import zeros
from scipy import weave

dx = 0.1
dy = 0.1
dx2 = dx*dx
dy2 = dy*dy

def py_update(u):
    nx, ny = u.shape
    for i in xrange(1,nx-1):
        for j in xrange(1, ny-1):
            u[i,j] = ((u[i+1, j] + u[i-1, j]) * dy2 +
                      (u[i, j+1] + u[i, j-1]) * dx2) / (2*(dx2+dy2))

def calc(N, Niter=100, func=py_update, args=()):
    u = zeros([N, N])
    u[0] = 1
    for i in range(Niter):
        func(u,*args)
    return u

In [ ]:
%timeit calc(100, 200, func=py_update)

Using `numpy`

In [ ]:
def num_update(u):
    u[1:-1,1:-1] = ((u[2:,1:-1]+u[:-2,1:-1])*dy2 + 
                    (u[1:-1,2:] + u[1:-1,:-2])*dx2) / (2*(dx2+dy2))

In [ ]:
%timeit calc(100, 200, func=num_update)

Using the `weave` module to implement C.

In [ ]:
def weave_update(u):
    code = """
    int i, j;
    for (i=1; i<Nu[0]-1; i++) {
       for (j=1; j<Nu[1]-1; j++) {
           U2(i,j) = ((U2(i+1, j) + U2(i-1, j))*dy2 + \
                       (U2(i, j+1) + U2(i, j-1))*dx2) / (2*(dx2+dy2));
       }
    }
    """
    weave.inline(code, ['u', 'dx2', 'dy2'])

In [ ]:
%timeit calc(100, 200, func=weave_update)

---
<a id='ctypes'></a>
#### Cython

![](http://www.ctcc.no/events/events-in-oslo/2012/cython-logo-507.jpg)

[Cython](http://cython.org/) serves as another interface between Python and C.  Strictly speaking, Cython is a static compiler for a superset of the Python language.  From the [documentation](http://docs.cython.org/):  "The Cython language is a superset of the Python language that additionally supports calling C functions and declaring C types on variables and class attributes. This allows the compiler to generate very efficient C code from Cython code."

Basically any Python code is valid Cython code, but Cython additionally imposes C data types (and manages them intelligently behind the scenes), as well as provides a basic superset of the language to manage this extensions (such as the keywords `cdef` and `cimport`).  So you lose the flexibility of a dynamic interpreted language, but gain some performance and interoperability.

- [Cython language reference](http://docs.cython.org/src/userguide/language_basics.html#language-basics)

##### Using Cython

Using Cython consists of the following steps:

1. Composing a `.pyx` source file.
1. Running the Cython compiler to generate a C file.
1. Running a C compiler to generate a compiled library.
1. Running the Python interpreter and `import`ing the module.

In [ ]:
%%file factorial.pyx
def factorial(n):
    return 1 if (n < 1) else n * factorial(n-1)

Hop over to a shell and take a look at the generated code in `factorial.c`.  I dare you to untangle it.

Given a `pyx` file, there are a few ways we can convert this to a C module using Cython.  Both of these require the presence of the `Python.h` development header, which may not be installed on your machine (instructions [here](https://stackoverflow.com/questions/21530577/fatal-error-python-h-no-such-file-or-directory)).  The first is to carry out the basic steps manually.

In [ ]:
!cython factorial.pyx

In [ ]:
#!gcc -shared -fPIC -o factorial.so factorial.c  # on own machine
!gcc -I /software/python3-3.4.1/include/python3.4m/ -shared -fPIC -o factorial.so factorial.c  # on EWS

In [ ]:
import factorial
print(factorial.factorial(6))

Another path to Cython is to use the built-in `cythonize` function.  (This is preferred for large codes.)  I had trouble getting this to work on the EWS installation of Python, however.

In [ ]:
from distutils.core import setup
from Cython.Build import cythonize

setup(
    ext_modules=cythonize("factorial.pyx"),
)
# problem is clang ñ gcc here (-fopenmp)

In [ ]:
%tb

In [ ]:
!python setup.py build_ext --inplace

When working with the IPython notebook environment, there is a convenient way of compiling and loading Cython code.  Using the `%%cython` IPython magic, we can simply type the Cython code in a code cell and let IPython take care of the conversion to C code, compilation and loading of the function.  To be able to use the `%%cython` magic in IPython, we first need to load the extension `cythonmagic`:

In [ ]:
%load_ext cythonmagic

In [ ]:
%%cython
cimport numpy

def cy_dcumsum2(numpy.ndarray[numpy.float64_t, ndim=1] a, numpy.ndarray[numpy.float64_t, ndim=1] b):
    cdef int i, n = len(a)
    b[0] = a[0]
    for i from 1 <= i < n:
        b[i] = b[i-1] + a[i]
    return b

In [ ]:
import numpy as np

x = np.ones((32))
y = np.zeros(x.shape)

cy_dcumsum2(x,y)

print(y)

Okay, so as a refresher, Cython usage [boils down to](http://docs.cython.org/src/userguide/numpy_tutorial.html#your-cython-environment):

    $ cython mymodule.pyx
$ gcc mymodule.c -o mymodule.so
    $ python
    >>> import mymodule

---
<a id='refs'></a>
## References

- [On using Arrays in FORTRAN](http://web.stanford.edu/class/me200c/tutorial_77/10_arrays.html)
- [Gently introducing C](http://www.ntu.edu.sg/home/ehchua/programming/cpp/c0_Introduction.html)
- [On ctypes](https://docs.python.org/2/library/ctypes.html)
- [On exploiting Python for performance](http://wiki.scipy.org/PerformancePython)
- [A look at combining Python with FORTRAN, C and C++](http://www.sam.math.ethz.ch/~raoulb/teaching/PythonTutorial/combining.html)

- J. R. Johansson's [Scientific Python Lectures](https://github.com/jrjohansson/scientific-python-lectures)—[Lesson 6A](http://nbviewer.ipython.org/urls/raw.github.com/jrjohansson/scientific-python-lectures/master/Lecture-6A-Fortran-and-C.ipynb)

- [Stack Overflow:  Wrapping a C library in Python: C, Cython or ctypes?](https://stackoverflow.com/questions/1942298/wrapping-a-c-library-in-python-c-cython-or-ctypes)

---
<a id='credits'></a>
## Credits

Lakshmi Rao and Neal Davis developed these materials for [Computational Science and Engineering](http://cse.illinois.edu/) at the University of Illinois at Urbana–Champaign.

<img src="http://i.creativecommons.org/l/by/3.0/88x31.png" align="left">
This content is available under a [Creative Commons Attribution 3.0 Unported License](https://creativecommons.org/licenses/by/3.0/).

[![](https://bytebucket.org/davis68/resources/raw/f7c98d2b95e961fae257707e22a58fa1a2c36bec/logos/baseline_cse_wdmk.png?token=be4cc41d4b2afe594f5b1570a3c5aad96a65f0d6)](http://cse.illinois.edu/)